In [2]:
import numpy as np
import pandas as pd

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,f1_score, precision_score, recall_score, roc_auc_score

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV

import pickle

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
#a

In [4]:
class uji(): 
    def __init__(self,data): 
        df0 = pd.read_csv('../data5/uji%snorm.csv'%data, delimiter= ",", header=None)
        df0=df0.fillna(0)
        self.com=df0[0]
        self.prot=df0[1282]
        self.x =df0.drop([0,1282], axis=1)
        df0[1300]=0
        
        self.y=df0[1300]
        self.dt=data
class latih(): 
    def __init__(self,data): 
        df = pd.read_csv("../data5/%s.csv"%data, delimiter=',')
        df.head()
        self.X = df.drop(['label'], axis=1)
        self.Y = df['label']
        self.dt=data
def dtuji(data): 
    return uji(data) 
def dtlatih(data): 
    return latih(data) 

dtf =dtuji('mbafika') #dtf.com, dtf.prot, dtf.x, dtf.y, dtf.dt
dtl =dtuji('mbalinda') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt
dtt =dtuji('tambahan_2') #dtl.com, dtl.prot, dtl.x, dtl.y dtl.dt
df1= dtlatih('hasilsmote_spike30_1') #df1.X,df1.Y,df1.dt
df2= dtlatih('hasilsmote_spike30_2') #df2.X,df2.Y,df2.dt
df3= dtlatih('hasilsmote_spike30_3') #df3.X,df3.Y,df3.dt
df4= dtlatih('hasilsmote_spike30_4') #df4.X,df4.Y,df4.dt
df5= dtlatih('hasilsmote_spike30_5') #df5.X,df5.Y,df5.dt

In [5]:
#df.X

In [6]:
scoring = {'AUC': 'roc_auc', 'Accuracy': 'accuracy','F-measure':'f1','Precision':'precision', 'Recall':'recall'}


In [7]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier

Using TensorFlow backend.


In [7]:
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(640, input_dim=1281, init='normal', activation='relu'))
    model.add(Dense(320, init='normal', activation='relu'))
    model.add(Dense(1, init='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
def mlp(df):
    mlp = MLPClassifier(max_iter=1000, batch_size=128,hidden_layer_sizes=(640,320,),activation= 'relu',
                        solver= 'adam', alpha= 0.5,learning_rate= 'constant')
    model = Sequential()
    model.add(Dense(640, input_dim=1281, init='uniform', activation='relu'))
    model.add(Dense(320, init='uniform', activation='relu'))
    model.add(Dense(1, init='uniform', activation='sigmoid'))
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_model, nb_epoch=100, batch_size=128, verbose=0,class_weight="balanced")))
    pipeline = Pipeline(estimators)
#    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
#    results = cross_val_score(pipeline, df.X, df.Y, cv=kfold)
#    print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))
    parameter_space = {
        'hidden_layer_sizes': [(640,320,)],
        'activation': ['relu'],
        'solver': ['adam'],
        'alpha': [0.5],
        'learning_rate': ['constant'],
    }
    mlp = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=5,scoring=scoring, refit='AUC',return_train_score=True)
    mlp.fit(df.X, df.Y)
    print('done_training')
    filename = '../data5/model/mlp_{0}'.format(df.dt)
    pickle.dump(mlp, open(filename, 'wb'))
    print('done_savemodel')
    return mlp

In [9]:
#rf5_100n_1000m.cv_results_#['mean_test_score']
#stds = clf.cv_results_['std_test_score']
def print_cv_results_mlp(model,o):
    rf = pickle.load(open('../data5/model/{0}_{1}'.format(model.__name__,o.dt), 'rb'))
    print('{0}_{1}'.format(model.__name__,o.dt))
    print('Best parameters found:', rf.best_params_)
    print('AUC :',rf.cv_results_['mean_test_AUC'])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'])
    print('Precision :',rf.cv_results_['mean_test_Precision'])
    print('Recall :',rf.cv_results_['mean_test_Recall'])

In [10]:
mlp1=mlp(df1)
mlp2=mlp(df2)
mlp3=mlp(df3)
mlp4=mlp(df4)
mlp5=mlp(df5)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(640, input_dim=1281, activation="relu", kernel_initializer="uniform")`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(320, activation="relu", kernel_initializer="uniform")`
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


done_training
done_savemodel


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(640, input_dim=1281, activation="relu", kernel_initializer="uniform")`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(320, activation="relu", kernel_initializer="uniform")`
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


done_training
done_savemodel


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(640, input_dim=1281, activation="relu", kernel_initializer="uniform")`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(320, activation="relu", kernel_initializer="uniform")`
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


done_training
done_savemodel


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(640, input_dim=1281, activation="relu", kernel_initializer="uniform")`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(320, activation="relu", kernel_initializer="uniform")`
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


done_training
done_savemodel


D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(640, input_dim=1281, activation="relu", kernel_initializer="uniform")`
  """
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(320, activation="relu", kernel_initializer="uniform")`
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="sigmoid", kernel_initializer="uniform")`
  import sys


done_training
done_savemodel


In [11]:
print_cv_results_mlp(mlp,df1)
print_cv_results_mlp(mlp,df2)
print_cv_results_mlp(mlp,df3)
print_cv_results_mlp(mlp,df4)
print_cv_results_mlp(mlp,df5)

mlp_hasilsmote_spike30_1
Best parameters found: {'activation': 'relu', 'alpha': 0.5, 'hidden_layer_sizes': (640, 320), 'learning_rate': 'constant', 'solver': 'adam'}
AUC : [0.98029619]
Accuracy : [0.94008613]
F-measure : [0.93317225]
Precision : [0.95326757]
Recall : [0.92696607]
mlp_hasilsmote_spike30_2
Best parameters found: {'activation': 'relu', 'alpha': 0.5, 'hidden_layer_sizes': (640, 320), 'learning_rate': 'constant', 'solver': 'adam'}
AUC : [0.98103999]
Accuracy : [0.94060392]
F-measure : [0.93033007]
Precision : [0.96650553]
Recall : [0.91483504]
mlp_hasilsmote_spike30_3
Best parameters found: {'activation': 'relu', 'alpha': 0.5, 'hidden_layer_sizes': (640, 320), 'learning_rate': 'constant', 'solver': 'adam'}
AUC : [0.97938448]
Accuracy : [0.93621216]
F-measure : [0.9259947]
Precision : [0.95765035]
Recall : [0.91561124]
mlp_hasilsmote_spike30_4
Best parameters found: {'activation': 'relu', 'alpha': 0.5, 'hidden_layer_sizes': (640, 320), 'learning_rate': 'constant', 'solver': 

In [9]:
from sklearn import svm

In [12]:
def svm_(df,kern):
    parameter_space0 = {
        'C': [0.03125, 0.5, 8, 128, 2048, 32768],
        'kernel': [kern],#'rbf', 'sigmoid', 'poly','linear'],
        'gamma' : [2, 0.125,  0.0078125,0.00048828125, 0.00003051757],
    }
    classifier0 = svm.SVC()
    classifier0 = GridSearchCV(classifier0, parameter_space0, scoring=scoring,refit='AUC',return_train_score=True, n_jobs=-1, cv=5)
    classifier0.fit(df.X, df.Y)
    print('done_training')
    print('Best parameters found:', classifier0.best_params_)
    filename = '../data5/model/svm_{0}_{1}'.format(kern,df.dt)
    pickle.dump(classifier0, open(filename, 'wb'))
    print('done_savemodel')
    return classifier0

In [13]:
def svm_cg(df,kern,c,g):
    parameter_space0 = {
        'C': [c],
        'kernel': [kern],#'rbf', 'sigmoid', 'poly','linear'],
        'gamma' : [g],
    }
    classifier0 = SVC(probability=True)
    classifier0 = GridSearchCV(classifier0, parameter_space0, scoring=scoring,refit='AUC',return_train_score=True, n_jobs=-1, cv=5)
    classifier0.fit(df.X, df.Y)
    print('done_training')
    print('Best parameters found:', classifier0.best_params_)
    filename = '../data5/model/svm_{0}_{1}'.format(kern,df.dt)
    pickle.dump(classifier0, open(filename, 'wb'))
    print('done_savemodel')
    return classifier0

In [14]:
def save_svm(svm1,df):
    filename = '../data5/model/svm_rbf_{0}'.format(df.dt)
    pickle.dump(svm1, open(filename, 'wb'))

In [15]:
svma=svm_cg(df1,'rbf',0.5, 0.0078125)
svmb=svm_cg(df2,'rbf',8, 0.0078125)
svmc=svm_cg(df3,'rbf', 0.5,  0.0078125)
svmd=svm_cg(df4,'rbf',8, 0.0078125)
svme=svm_cg(df5,'rbf',0.5, 0.0078125)

done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel


In [13]:
svme=svm_cg(df5,'rbf',0.5, 0.0078125)

done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel


In [57]:
save_svm(svm1,df1)
save_svm(svm2,df2)
save_svm(svm3,df3)
save_svm(svm4,df4)
save_svm(svm5,df5)

In [ ]:
svm1=svm_(df1,'rbf')
svm2=svm_(df2,'rbf')
svm3=svm_(df3,'rbf')
svm4=svm_(df4,'rbf')
svm5=svm_(df5,'rbf')

done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel
done_training
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
done_savemodel


In [12]:
svm5=svm_(df5,'rbf')

NameError: name 'svm_' is not defined

In [8]:
#rf5_100n_1000m.cv_results_#['mean_test_score']
#stds = clf.cv_results_['std_test_score']
def print_cv_results_svm(o):
    rf = pickle.load(open('../data5/model/svm_rbf_{0}'.format(o.dt), 'rb'))
    print('svm_rbf_{0}'.format(o.dt))
    print('Best parameters found:', rf.best_params_)
    print('AUC :',rf.cv_results_['mean_test_AUC'][(rf.cv_results_['rank_test_AUC']-1)[0]])
    print('Accuracy :',rf.cv_results_['mean_test_Accuracy'][(rf.cv_results_['rank_test_Accuracy']-1)[0]])
    print('F-measure :',rf.cv_results_['mean_test_F-measure'][(rf.cv_results_['rank_test_F-measure']-1)[0]])
    print('Precision :',rf.cv_results_['mean_test_Precision'][(rf.cv_results_['rank_test_Precision']-1)[0]])
    print('Recall :',rf.cv_results_['mean_test_Recall'][(rf.cv_results_['rank_test_Recall']-1)[0]])

In [9]:
print_cv_results_svm(df1)
print_cv_results_svm(df2)
print_cv_results_svm(df3)
print_cv_results_svm(df4)
print_cv_results_svm(df5)

svm_rbf_hasilsmote_spike30_1
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9809299577262964
Accuracy : 0.9274313010144081
F-measure : 0.9212845427109126
Precision : 0.9293121025982527
Recall : 0.9256734183545886
svm_rbf_hasilsmote_spike30_2
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9778508653864584
Accuracy : 0.9384077916401846
F-measure : 0.9271187225262867
Precision : 0.9648776821309148
Recall : 0.912514128532133
svm_rbf_hasilsmote_spike30_3
Best parameters found: {'C': 0.5, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9809816877130565
Accuracy : 0.9274313844223914
F-measure : 0.9203157567986583
Precision : 0.9332633264987387
Recall : 0.9212859881637077
svm_rbf_hasilsmote_spike30_4
Best parameters found: {'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}
AUC : 0.9789307361293382
Accuracy : 0.9386661895730011
F-measure : 0.9271313781300113
Precision : 0.9649067543553904
Recall : 0.912773526715012
svm_rbf_hasilsmote_spike30

In [19]:

def prediksi(md,model,n,m,p,data,d):
    pred_x = pd.DataFrame(md.predict_proba(data.x))
    prob_x = pd.DataFrame(np.where(pred_x >= p, 1,0))
    print("threshold {0} ".format(p))
    print("model_hasilsmote_{0} ".format(d))
    print("hasil prediksi ",data.dt," : (0,1)")
    print(confusion_matrix(data.y, prob_x[1]))
    pred = pd.DataFrame(prob_x)
    pred[2] = data.com
    pred[3] = data.prot
    #print(pred.head())
    #print(prob_x)
    pred[4] = pred_x[1]
    #print(pred.head())
    #print(pd.DataFrame(prob_x)[1].head())
    pred.to_excel('../data3/hasil_pred/{0}_{1}p_{2}_modelhasilsmote{3}.xlsx'.format(model.__name__,p,data.dt,d))

In [23]:
def fitn2pred(model,n,m,p,datalatih,datauji1,datauji2,datauji3,d):
    md=print_cv_results(model,n,m,datalatih)
    prediksi(md,model,n,m,p,datauji1,d)
    prediksi(md,model,n,m,p,datauji2,d)
    prediksi(md,model,n,m,p,datauji3,d)

In [25]:
#fitn2pred(mlp,100,1000,0.5,df1,dtf,dtl,dtg,1)